In [1]:
from pya3 import *
import datetime as dt
import time

In [2]:
_id='308479'
apikey = '15NCRnTP02gQjlVS4C3EN0p6fhLVePQmD1xRhqKmMKf1EfSfQW5JIs74WKaZX74nXA2MPlU4574hOMd3DlhM4EWAOL6O3sq5M0X6EWh1XMHpQyB2kttXedHiEXqptCB5'

In [3]:
alice = Aliceblue(user_id=_id,api_key=apikey)

In [4]:
sessionid = alice.get_session_id() # Get Session ID

In [5]:
print(sessionid)

{'stat': 'Ok', 'sessionID': 'y8slLngS1tDdVV5j4GBxxJ5TPt6qp6avsEf2xKdhkT2XR67qMPXAIZATz8IbO33IuX6iJO5mQdWqrtBbAdkR3SaLj1zOWWdgV1INZKyxPbGqG2YzxASiXMC307lzmXiKVSIhQZ9NKl458dbAlifYFmEDGcxUo8hmjE2x54S3kOhAKsCiK0Etp4dv7dHTyE0xzJI7qJ3tXdLSHgfZZq7AEM3SQTctZgZN3aKXper744tSFd8FUrd6vOgrgB4jxioQ'}


In [6]:
banknifty = alice.get_instrument_by_symbol('INDICES','NIFTY BANK')

In [7]:
###### Check trade Function
def check_trade():
    alice=Aliceblue(_id,apikey)
    alice.get_session_id()['sessionID']
    days=1
    exchange='INDICES'
    spot_symbol='NIFTY BANK'
    interval='D'
    indices=True
    from_date= dt.datetime.now()- dt.timedelta(days=days)
    to_date=datetime.now()
    token=banknifty
    data=alice.get_historical(token,from_date,to_date,interval,indices)
    y_high = data['high'][0]
    y_low = data['low'][0]
    today_open = data['open'][1]
    if(y_high > today_open and today_open > y_low):
        return True
    else:
        print("No Trade today")
        return False

In [8]:
Trade = check_trade()
print(Trade)

True


In [9]:
def get_curr_expiry(spot_ltp):
    spot_ltp = int(spot_ltp)
    print(f"Current expiry for {spot_ltp}")
    global datecale
    datecale = dt.date.today()
    while(1):
        expiry = alice.get_instrument_for_fno(exch="NFO",symbol='BANKNIFTY', expiry_date=str(datecale), is_fut=False,strike=spot_ltp, is_CE=True)
        #print(expiry)
        try:
            if list(expiry.values())[0] == 'Not_ok':
                print(f"{datecale} is not expiry")
                datecale = datecale + dt.timedelta(days=1)
            else:
                return datecale
                break
        except:
            return datecale

In [12]:
# get and Round off ltp on 09:20
# Entering into Trade
while(1):
    if dt.datetime.now().time()>=dt.time(9,18,5):
        Trade = check_trade()
        print(Trade)
    if (Trade == True):
        print(dt.datetime.now().time())
        if dt.datetime.now().time()>=dt.time(9,20,5):
            bnf_spot_ltp = float(alice.get_scrip_info(banknifty)['LTP'])
            round_bnf_spot_ltp = round((bnf_spot_ltp/100),0)*100
            bnf_expiry_date = get_curr_expiry(round_bnf_spot_ltp)
            ce_instrument = alice.get_instrument_for_fno(exch="NFO",symbol='BANKNIFTY', expiry_date=str(bnf_expiry_date),
                                                         is_fut=False,strike=round_bnf_spot_ltp, is_CE=True)
            pe_instrument = alice.get_instrument_for_fno(exch="NFO",symbol='BANKNIFTY', expiry_date=str(bnf_expiry_date),
                                                         is_fut=False,strike=round_bnf_spot_ltp, is_CE=False)
            order1 = {  "instrument"        : ce_instrument,
            "order_type"        : OrderType.Market,
            "quantity"          : 25,
            "transaction_type"  : TransactionType.Sell,
            "product_type"      : ProductType.Intraday,
            "order_tag"         : "Order1"}
            order2 = {  "instrument"        : pe_instrument,
            "order_type"        : OrderType.Market,
            "quantity"          : 25,
            "transaction_type"  : TransactionType.Sell,
            "product_type"      : ProductType.Intraday,
            "order_tag"         : "Order2"}
            orders = [order1, order2]
            brorder = alice.place_basket_order(orders)
            print("Order placed")
            print(f"LTP is {bnf_spot_ltp} Sell PE and CE on {round_bnf_spot_ltp} Price at expiry {bnf_expiry_date}.")
            #print(ce_instrument,pe_instrument)
            break
        else:
            time.sleep(1)
    else:
        print("Not Trade today")
        break

09:17:37.005454
09:17:38.010744
09:17:39.019486
09:17:40.022718
09:17:41.030402
09:17:42.040418
09:17:43.042424
09:17:44.045795
09:17:45.059856
09:17:46.060877
09:17:47.062423
09:17:48.070296
09:17:49.084843
09:17:50.086687
09:17:51.096695
09:17:52.101425
09:17:53.113541
09:17:54.115004
09:17:55.121837
09:17:56.134203
09:17:57.139870
09:17:58.149970
09:17:59.163734
09:18:00.176514
09:18:01.189240
09:18:02.200534
09:18:03.211565
09:18:04.217686
True
09:18:06.189361
True
09:18:08.045700
True
09:18:10.384834
True
09:18:13.004302
True
09:18:14.998406
True
09:18:16.798155
True
09:18:18.875973
True
09:18:20.821171
True
09:18:22.782371
True
09:18:24.823044
True
09:18:26.759432
True
09:18:28.702717
True
09:18:30.651238
True
09:18:32.607189
True
09:18:35.387479
True
09:18:37.406048
True
09:18:39.351336
True
09:18:41.301148
True
09:18:43.269235
True
09:18:45.198838
True
09:18:47.134393
True
09:18:49.086065
True
09:18:51.237872
True
09:18:53.746929
True
09:18:55.741977
True
09:18:57.685721
True
0

In [13]:
## Calculating the Target and the StopLoss
cedict = alice.get_order_history(brorder[0]['NOrdNo'])
pedict = alice.get_order_history(brorder[1]['NOrdNo'])
cesell = float(alice.get_scrip_info(ce_instrument)['LTP'])#float(cedict['Prc'])
pesell = float(alice.get_scrip_info(pe_instrument)['LTP'])#float(pedict['Prc'])
target = (cesell + pesell) * 0.9
stoploss = (cesell + pesell) * 1.10
print("Target: ", target)
print("Stoploss: ",stoploss)
print("Value Sum: ",cesell + pesell)

Target:  521.325
Stoploss:  637.1750000000001
Value Sum:  579.25


In [18]:
## Calculating the trade 
while(1):
    time.sleep(5)
    celtp = float(alice.get_scrip_info(ce_instrument)['LTP'])
    peltp = float(alice.get_scrip_info(pe_instrument)['LTP'])
    if ((celtp + peltp) <= target):
        print("Target Achive Square Off the Trade at: ", celtp + peltp)
        order3 = {  "instrument"        : ce_instrument,
        "order_type"        : OrderType.Market,
        "quantity"          : 25,
        "transaction_type"  : TransactionType.Buy,
        "product_type"      : ProductType.Intraday,
        "order_tag"         : "Order3"}
        order4 = {  "instrument"        : pe_instrument,
        "order_type"        : OrderType.Market,
        "quantity"          : 25,
        "transaction_type"  : TransactionType.Buy,
        "product_type"      : ProductType.Intraday,
        "order_tag"         : "Order4"}
        orders = [order3, order4]
        brorder = alice.place_basket_order(orders)
        break
    elif (celtp + peltp) >= stoploss:
        print("StopLoss Hit Square Off the Trade")
        print("Target Achive Square Off the Trade at: ", celtp + peltp)
        order3 = {  "instrument"        : ce_instrument,
        "order_type"        : OrderType.Market,
        "quantity"          : 25,
        "transaction_type"  : TransactionType.Buy,
        "product_type"      : ProductType.Intraday,
        "order_tag"         : "Order3"}
        order4 = {  "instrument"        : pe_instrument,
        "order_type"        : OrderType.Market,
        "quantity"          : 25,
        "transaction_type"  : TransactionType.Buy,
        "product_type"      : ProductType.Intraday,
        "order_tag"         : "Order4"}
        orders = [order3, order4]
        brorder = alice.place_basket_order(orders)
        break
    else:
        print(f"Current Sum: {celtp + peltp} and Running Profit/Loss: {(cesell + pesell)-(celtp + peltp)}")

Current Sum: 563.1500000000001 and Running Profit/Loss: 16.09999999999991
Current Sum: 563.55 and Running Profit/Loss: 15.700000000000045
Current Sum: 562.95 and Running Profit/Loss: 16.299999999999955
Current Sum: 562.15 and Running Profit/Loss: 17.100000000000023
Current Sum: 563.35 and Running Profit/Loss: 15.899999999999977
Current Sum: 563.35 and Running Profit/Loss: 15.899999999999977
Current Sum: 565.15 and Running Profit/Loss: 14.100000000000023
Current Sum: 564.55 and Running Profit/Loss: 14.700000000000045
Current Sum: 566.0 and Running Profit/Loss: 13.25
Current Sum: 565.35 and Running Profit/Loss: 13.899999999999977
Current Sum: 564.1 and Running Profit/Loss: 15.149999999999977
Current Sum: 564.55 and Running Profit/Loss: 14.700000000000045
Current Sum: 566.7 and Running Profit/Loss: 12.549999999999955
Current Sum: 564.2 and Running Profit/Loss: 15.049999999999955
Current Sum: 567.05 and Running Profit/Loss: 12.200000000000045
Current Sum: 567.0 and Running Profit/Loss: 12.

Current Sum: 563.9 and Running Profit/Loss: 15.350000000000023
Current Sum: 563.85 and Running Profit/Loss: 15.399999999999977
Current Sum: 561.8499999999999 and Running Profit/Loss: 17.40000000000009
Current Sum: 562.15 and Running Profit/Loss: 17.100000000000023
Current Sum: 562.0 and Running Profit/Loss: 17.25
Current Sum: 563.8 and Running Profit/Loss: 15.450000000000045
Current Sum: 562.9 and Running Profit/Loss: 16.350000000000023
Current Sum: 563.85 and Running Profit/Loss: 15.399999999999977
Current Sum: 564.5 and Running Profit/Loss: 14.75
Current Sum: 564.65 and Running Profit/Loss: 14.600000000000023
Current Sum: 564.3 and Running Profit/Loss: 14.950000000000045
Current Sum: 562.3 and Running Profit/Loss: 16.950000000000045
Current Sum: 563.15 and Running Profit/Loss: 16.100000000000023
Current Sum: 564.2 and Running Profit/Loss: 15.049999999999955
Current Sum: 564.85 and Running Profit/Loss: 14.399999999999977
Current Sum: 562.8499999999999 and Running Profit/Loss: 16.40000

Current Sum: 566.55 and Running Profit/Loss: 12.700000000000045
Current Sum: 565.7 and Running Profit/Loss: 13.549999999999955
Current Sum: 566.0 and Running Profit/Loss: 13.25
Current Sum: 564.95 and Running Profit/Loss: 14.299999999999955
Current Sum: 566.4 and Running Profit/Loss: 12.850000000000023
Current Sum: 564.9 and Running Profit/Loss: 14.350000000000023
Current Sum: 565.25 and Running Profit/Loss: 14.0
Current Sum: 566.3 and Running Profit/Loss: 12.950000000000045
Current Sum: 565.55 and Running Profit/Loss: 13.700000000000045
Current Sum: 565.4 and Running Profit/Loss: 13.850000000000023
Current Sum: 566.65 and Running Profit/Loss: 12.600000000000023
Current Sum: 566.4 and Running Profit/Loss: 12.850000000000023
Current Sum: 567.55 and Running Profit/Loss: 11.700000000000045
Current Sum: 566.8499999999999 and Running Profit/Loss: 12.400000000000091
Current Sum: 568.1 and Running Profit/Loss: 11.149999999999977
Current Sum: 567.35 and Running Profit/Loss: 11.899999999999977


Current Sum: 549.9 and Running Profit/Loss: 29.350000000000023
Current Sum: 551.85 and Running Profit/Loss: 27.399999999999977
Current Sum: 551.65 and Running Profit/Loss: 27.600000000000023
Current Sum: 549.65 and Running Profit/Loss: 29.600000000000023
Current Sum: 549.1500000000001 and Running Profit/Loss: 30.09999999999991
Current Sum: 550.6500000000001 and Running Profit/Loss: 28.59999999999991
Current Sum: 550.0 and Running Profit/Loss: 29.25
Current Sum: 548.3 and Running Profit/Loss: 30.950000000000045
Current Sum: 549.0 and Running Profit/Loss: 30.25
Current Sum: 550.05 and Running Profit/Loss: 29.200000000000045
Current Sum: 549.95 and Running Profit/Loss: 29.299999999999955
Current Sum: 549.85 and Running Profit/Loss: 29.399999999999977
Current Sum: 549.45 and Running Profit/Loss: 29.799999999999955
Current Sum: 551.65 and Running Profit/Loss: 27.600000000000023
Current Sum: 551.1500000000001 and Running Profit/Loss: 28.09999999999991
Current Sum: 549.3 and Running Profit/Lo

Current Sum: 551.55 and Running Profit/Loss: 27.700000000000045
Current Sum: 553.25 and Running Profit/Loss: 26.0
Current Sum: 552.0999999999999 and Running Profit/Loss: 27.15000000000009
Current Sum: 552.4 and Running Profit/Loss: 26.850000000000023
Current Sum: 550.55 and Running Profit/Loss: 28.700000000000045
Current Sum: 551.15 and Running Profit/Loss: 28.100000000000023
Current Sum: 552.45 and Running Profit/Loss: 26.799999999999955
Current Sum: 554.25 and Running Profit/Loss: 25.0
Current Sum: 552.3 and Running Profit/Loss: 26.950000000000045
Current Sum: 552.95 and Running Profit/Loss: 26.299999999999955
Current Sum: 552.5 and Running Profit/Loss: 26.75
Current Sum: 553.95 and Running Profit/Loss: 25.299999999999955
Current Sum: 552.7 and Running Profit/Loss: 26.549999999999955
Current Sum: 551.1 and Running Profit/Loss: 28.149999999999977
Current Sum: 554.0 and Running Profit/Loss: 25.25
Current Sum: 552.8 and Running Profit/Loss: 26.450000000000045
Current Sum: 552.95 and Run

Current Sum: 544.2 and Running Profit/Loss: 35.049999999999955
Current Sum: 538.2 and Running Profit/Loss: 41.049999999999955
Current Sum: 540.0 and Running Profit/Loss: 39.25
Current Sum: 543.4 and Running Profit/Loss: 35.85000000000002
Current Sum: 543.15 and Running Profit/Loss: 36.10000000000002
Current Sum: 542.3499999999999 and Running Profit/Loss: 36.90000000000009
Current Sum: 543.45 and Running Profit/Loss: 35.799999999999955
Current Sum: 542.9 and Running Profit/Loss: 36.35000000000002
Current Sum: 542.5 and Running Profit/Loss: 36.75
Current Sum: 543.2 and Running Profit/Loss: 36.049999999999955
Current Sum: 542.3 and Running Profit/Loss: 36.950000000000045
Current Sum: 541.6 and Running Profit/Loss: 37.64999999999998
Current Sum: 539.95 and Running Profit/Loss: 39.299999999999955
Current Sum: 541.15 and Running Profit/Loss: 38.10000000000002
Current Sum: 539.25 and Running Profit/Loss: 40.0
Current Sum: 541.2 and Running Profit/Loss: 38.049999999999955
Current Sum: 539.9 an

Current Sum: 548.25 and Running Profit/Loss: 31.0
Current Sum: 547.2 and Running Profit/Loss: 32.049999999999955
Current Sum: 547.5 and Running Profit/Loss: 31.75
Current Sum: 546.2 and Running Profit/Loss: 33.049999999999955
Current Sum: 543.7 and Running Profit/Loss: 35.549999999999955
Current Sum: 545.3 and Running Profit/Loss: 33.950000000000045
Current Sum: 545.3 and Running Profit/Loss: 33.950000000000045
Current Sum: 545.75 and Running Profit/Loss: 33.5
Current Sum: 546.45 and Running Profit/Loss: 32.799999999999955
Current Sum: 546.15 and Running Profit/Loss: 33.10000000000002
Current Sum: 546.1500000000001 and Running Profit/Loss: 33.09999999999991
Current Sum: 545.15 and Running Profit/Loss: 34.10000000000002
Current Sum: 545.5 and Running Profit/Loss: 33.75
Current Sum: 543.6 and Running Profit/Loss: 35.64999999999998
Current Sum: 545.2 and Running Profit/Loss: 34.049999999999955
Current Sum: 544.3 and Running Profit/Loss: 34.950000000000045
Current Sum: 543.55 and Running P

Current Sum: 536.9 and Running Profit/Loss: 42.35000000000002
Current Sum: 537.4 and Running Profit/Loss: 41.85000000000002
Current Sum: 534.9 and Running Profit/Loss: 44.35000000000002
Current Sum: 536.3 and Running Profit/Loss: 42.950000000000045
Current Sum: 536.85 and Running Profit/Loss: 42.39999999999998
Current Sum: 535.9 and Running Profit/Loss: 43.35000000000002
Current Sum: 536.7 and Running Profit/Loss: 42.549999999999955
Current Sum: 536.75 and Running Profit/Loss: 42.5
Current Sum: 536.45 and Running Profit/Loss: 42.799999999999955
Current Sum: 536.25 and Running Profit/Loss: 43.0
Current Sum: 536.5 and Running Profit/Loss: 42.75
Current Sum: 534.85 and Running Profit/Loss: 44.39999999999998
Current Sum: 533.35 and Running Profit/Loss: 45.89999999999998
Current Sum: 534.4 and Running Profit/Loss: 44.85000000000002
Current Sum: 534.45 and Running Profit/Loss: 44.799999999999955
Current Sum: 534.25 and Running Profit/Loss: 45.0
Current Sum: 528.8 and Running Profit/Loss: 50.

Current Sum: 525.6 and Running Profit/Loss: 53.64999999999998
Current Sum: 523.7 and Running Profit/Loss: 55.549999999999955
Current Sum: 525.25 and Running Profit/Loss: 54.0
Current Sum: 525.7 and Running Profit/Loss: 53.549999999999955
Current Sum: 524.35 and Running Profit/Loss: 54.89999999999998
Current Sum: 523.6 and Running Profit/Loss: 55.64999999999998
Current Sum: 522.4 and Running Profit/Loss: 56.85000000000002
Current Sum: 524.1 and Running Profit/Loss: 55.14999999999998
Current Sum: 523.25 and Running Profit/Loss: 56.0
Current Sum: 524.7 and Running Profit/Loss: 54.549999999999955
Current Sum: 526.15 and Running Profit/Loss: 53.10000000000002
Current Sum: 526.0 and Running Profit/Loss: 53.25
Current Sum: 524.9 and Running Profit/Loss: 54.35000000000002
Current Sum: 523.2 and Running Profit/Loss: 56.049999999999955
Current Sum: 523.6 and Running Profit/Loss: 55.64999999999998
Current Sum: 524.85 and Running Profit/Loss: 54.39999999999998
Current Sum: 524.3499999999999 and Ru

In [19]:
### 272 Condition > No order
alice.get_scrip_info(ce_instrument)

{'optiontype': 'CE',
 'SQty': 50,
 'vwapAveragePrice': '245.25',
 'LTQ': '25',
 'Ltp': '239.30',
 'LTP': '239.30',
 'DecimalPrecision': 2,
 'openPrice': '311.15',
 'BRate': '238.90',
 'defmktproval': '3',
 'BQty': 50,
 'symbolname': 'BANKNIFTY',
 'noMktPro': '0',
 'LTT': '07/07/2023 14:47:29',
 'mktpro': '1',
 'TickSize': '5',
 'Multiplier': 1,
 'strikeprice': '45100.00',
 'TotalSell': '639600',
 'High': '374.95',
 'stat': 'Ok',
 'BodLotQty': 25,
 'yearlyHighPrice': '374.95',
 'yearlyLowPrice': '00.00',
 'exchFeedTime': '07-Jul-2023 14:47:29',
 'PrvClose': '436.85',
 'SRate': '239.25',
 'Change': '-197.55',
 'Series': 'XX',
 'TotalBuy': '408750',
 'Low': '175.00',
 'UniqueKey': 'NA',
 'PerChange': '-45.22',
 'companyname': '',
 'TradeVolume': '58924425',
 'TSymbl': 'BANKNIFTY2371345100CE',
 'Exp': '13 Jul, 2023',
 'LTD': 'NA'}